# Importing libraries

In [1]:
# Libraries
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../src'))

# Modules
from modules.ingesting import ingesting_PDF
from modules.vector_store_module    import chunk_documents, add_to_vector_store

# Retrival (summirising):
from langchain.prompts                  import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers      import StrOutputParser
from langchain_core.runnables           import RunnablePassthrough
from langchain_ollama                   import ChatOllama
from langchain.retrievers.multi_query   import MultiQueryRetriever

# Loading docs

In [2]:
print('Loading the document...')
documents = ingesting_PDF('../data/PDFs/Sustainable_development_of_distance_learning_in_continuing_adult_education__The impact_of_artificial_intelligence.pdf')

Loading the document...
Document loaded successfully!!!


# Embedding process
- ```Chunking```
- Adding to the database ```(Chroma)```
- ```RAG (retrival - Summirisation)```

## Chunking

In [3]:
# Chunk the documents
chunks = chunk_documents(documents)

Chunking the document...
Document chunked successfully!!!


## Add chunks to the vector store

In [4]:
vector_store = add_to_vector_store(chunks)

Adding to vector database...
Document added to vector database successfully!!!


## RAG (retrival - Summirisation)

In [5]:
# LLM from Ollama
llm = ChatOllama(model = 'gemma2') # gemma2 is faster, but llama3.2 is more comprehe

### Prompt

In [6]:
prompt = PromptTemplate(
    input_variables=['question'],
    template = """You are an AI assistant. Your task is to summarise the text coming from the input PDF, providing a
    summary in the form of a paragraph. The summary should be a concise and coherent representation of the main ideas of the text, including the 
    analysis of the text where the main points and the key arguments are shown. Additionally, the user will prompt a question and the second 
    task you will do after providing a summary is to generate five different versions of the given questions to retrieve the relevant documents 
    from a vector database. By generating multiple perspectives on the user's question, you can help the user to explore the text from 
    different angles, aiming to help the final goal of overcoming the limitations of the distance-based similarity search. Provide these 
    alternative questions separated by a new line. Original question: {question}""",
)

### Retrival

In [7]:
retriever = MultiQueryRetriever.from_llm(
    vector_store.as_retriever(),
    llm,
    prompt = prompt,
)

### RAG prompt (to answer only on the given context):

In [8]:
template = """Summarise and Answer the questions ONLY on the following context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

### Chain (Joining all the answers)

In [9]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Running the chain (Chatting):

In this section, to enable interactivity, the following command can be used:

```response = chain.invoke({"question": input("Enter the question:\n")})```

However, for the sake of simplicity, the question is provided directly in the notebook.

## Question 1

In [10]:
response = chain.invoke('What is the central theme of the paper?')
print(response)

The central theme of the paper is the **impact of artificial intelligence (AI) on the sustainable development of distance learning in continuing adult education.** 

The authors explore how AI-powered tools enhance personalization, interaction, and accessibility in online learning, ultimately contributing to a more inclusive and effective educational experience for adults. They also highlight the alignment of AI integration with the principles of sustainable development, emphasizing its role in reducing inequalities and promoting lifelong learning.  



## Question 2

In [11]:
response = chain.invoke('How is AI changing adult education?')
print(response)

This text excerpt highlights several ways AI is changing adult education:

* **Personalized Learning:** AI tailors content and assessments to individual learning styles and paces, making education more effective and engaging. 
* **Enhanced Interaction:**  AI enables both synchronous (real-time) and asynchronous communication, improving student engagement and interaction with educators.
* **Real-Time Feedback:**  AI provides immediate feedback on assignments and progress, helping learners identify areas for improvement and stay motivated.
* **Accessibility:** AI makes educational resources more accessible to a wider range of learners, regardless of their background or circumstances.
* **Increased Efficiency:** AI automates administrative tasks for educators, freeing up their time to focus on teaching and student interaction.

**Overall, AI is making adult education more personalized, engaging, efficient, and accessible.**


Let me know if you have any other questions about the text! 



## Question 3

In [12]:
response = chain.invoke('What is "adaptive learning" in AI contexts?')
print(response)

While the provided text discusses AI's impact on distance learning and mentions personalization, it doesn't explicitly define "adaptive learning." 

However, based on the context and general understanding of AI in education, we can infer that **adaptive learning** refers to a system where:

* **AI algorithms analyze learner data:** This data could include performance on quizzes, time spent on tasks, areas of difficulty, and even learning styles.
* **The learning experience adjusts accordingly:** Based on the analyzed data, the AI system tailors the content, pace, and difficulty level to each individual learner's needs. 

Essentially, adaptive learning uses AI to create a personalized learning path for each student, ensuring they are challenged appropriately and receive targeted support where needed.  


Let me know if you have any other questions! 



## Question 4

In [13]:
response = chain.invoke('How can institutions ensure ethical AI use?')
print(response)

The text you provided highlights the importance of ethical considerations in using AI in education, but it doesn't offer specific steps institutions can take to ensure ethical AI use. 

Here are some ways institutions can work towards ethical AI implementation:

**1. Develop a Clear AI Ethics Policy:**

* **Define Principles:** Establish guiding principles for AI development and deployment that prioritize fairness, transparency, accountability, privacy, and human well-being.
* **Data Governance:** Implement robust data governance policies that address data collection, storage, use, and sharing practices. Ensure compliance with relevant privacy regulations.
* **Bias Mitigation:**  Actively work to identify and mitigate potential biases in AI algorithms and datasets. This may involve using diverse datasets, employing bias detection tools, and involving stakeholders from different backgrounds in the development process.

**2.  Promote Transparency and Explainability:**

* **Explainable AI

## Question 5

In [14]:
response = chain.invoke('How is AI reshaping teacher roles?')
print(response)

While the text you provided doesn't directly address how AI reshapes teacher roles, we can infer some possibilities based on its discussion of AI in education.

Here are some ways AI could reshape teacher roles:

* **From direct instruction to personalized guidance:** AI-powered systems can handle repetitive tasks like delivering basic information or grading simple assignments, freeing up teachers to focus on individualized instruction, mentoring, and addressing complex learning needs. 
* **Data-driven insights for tailored support:** AI algorithms can analyze student performance data to identify strengths, weaknesses, and learning patterns. Teachers can leverage these insights to create personalized learning plans and interventions.
* **Enhanced collaboration and communication:** AI tools can facilitate communication between teachers and students, automate administrative tasks, and connect educators with professional development resources, enabling them to collaborate more effectively

## Question 6

In [15]:
response = chain.invoke('What is the role of educators in AI-integrated systems?')
print(response)

The text highlights several roles educators play in AI-integrated distance learning systems:

* **Focus on Teaching and Interaction:** AI can automate administrative tasks and provide data analytics, freeing up educators to focus more on teaching and student interaction. 
* **Personalized Learning Support:** Educators can utilize AI's personalized learning capabilities to tailor content and assessments to individual student needs.
* **Leveraging Feedback Mechanisms:** Real-time feedback provided by AI systems can be used by educators to guide student learning and address areas needing improvement.

In essence, AI augments educators' roles, allowing them to become more facilitators of learning, mentors, and guides rather than solely task managers and content deliverers. 


Let me know if you have any other questions about the text!
